<a href="https://colab.research.google.com/github/TiltedCoding/Thesis_/blob/main/File_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 42.3 MB/s 
     |████████████████████████████████| 163 kB 61.9 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.4 MB/s 
     |████████████████████████████████| 115 kB 57.6 MB/s 
     |████████████████████████████████| 212 kB 43.2 MB/s 
     |████████████████████████████████| 127 kB 54.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.3 MB/s 


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np
import torch
import nltk
import re
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
path = '/content/IS1001c.transcript.txt'

In [7]:
file = open(path, 'r')

In [8]:
text = file.read().strip()

In [9]:
text

"Okay. So we are here for the concept design meeting. So, we will first start by summarizing the mm the previous meeting and the decision we've taken. Uh I will take notes during this meeting so uh that you can uh look at my uh folder to see the summary of this meeting afterwards. So each of you will uh show us the various investigation they've done during uh previous uh hours. We'll then t take decision in concert and then uh uh we will uh define the nest next task, to have to be done before the next meeting. So, last time we decided to have a simple interface. We also decided to have a wheel to change channel previous channel button. Channel digit uh buttons should be uh protected by plastic cover or something for the remote control to look very simple. We have also button for volume, and to switch on off the T_V_. We have also uh the lightening feature for the remote control to be easy to find, and for fast development and low cost we have decided to have no L_C_D_ no voice features

In [10]:
regex='(\S*)[:]'
summ_mod = "knkarthick/MEETING_SUMMARY"
sent_mod = f"cardiffnlp/twitter-roberta-base-sentiment"
summ_new = ''
length = 0
comblist = []
inputs = []
iter = 0
RobPos = []
RobNeg = []
RobNeu = []

In [11]:
def comb(text):
  summ_mod = "knkarthick/MEETING_SUMMARY"
  sent_mod = f"cardiffnlp/twitter-roberta-base-sentiment"
  tokenizer = AutoTokenizer.from_pretrained(summ_mod)
  model = AutoModelForSeq2SeqLM.from_pretrained(summ_mod)
  maxtokens = tokenizer.max_len_single_sentence
  length = 0
  comb = ''
  comblist = []
  iter = 0
  sentences = nltk.tokenize.sent_tokenize(text)
  for sentence in sentences:
    iter+= 1
    total_len = len(tokenizer.tokenize(sentence)) + length
    if total_len < maxtokens:
      comb += sentence
      length = total_len
      if iter == len(sentences):
        comblist.append(comb.strip())
    else:
      comblist.append(comb.strip())
      comb = sentence + " "
      length = len(tokenizer.tokenize(sentence))
  inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
  summ_input = []
  for input in inputs:
    output = model.generate(**input)
    summ_input.append(tokenizer.decode(*output, skip_special_tokens=True))
  summ_new = ''
  for i in range(len(summ_input)):
    try:
      text = summ_input[i]
      summ_new = summ_new + ' ' + text
    except:
      print(f'Broke for id {i}')
  return summ_new, len(comblist)

In [12]:
def summary(text):
  summ_mod = "knkarthick/MEETING_SUMMARY"
  sent_mod = f"cardiffnlp/twitter-roberta-base-sentiment"
  tokenizer = AutoTokenizer.from_pretrained(summ_mod)
  model = AutoModelForSeq2SeqLM.from_pretrained(summ_mod)
  maxtokens = tokenizer.max_len_single_sentence
  summ_new = comb(text) [0]
  first_summary = summ_new
  len_comblist = comb(text)[1]
  while len_comblist != 1:
    summ_new = comb(summ_new)[0]
    len_comblist = comb(summ_new)[1]
  # input = tokenizer(summ_new, return_tensors='pt')
  # output = model.generate(**input)
  # final = tokenizer.decode(*output, skip_special_tokens=True)
  final = summ_new
  return final, first_summary

In [13]:
def sentiment(text):
  tokenizer = AutoTokenizer.from_pretrained(summ_mod)
  model = AutoModelForSeq2SeqLM.from_pretrained(summ_mod)
  maxtokens = tokenizer.max_len_single_sentence
  length = 0
  comb = ''
  comblist = []
  iter = 0
  sentences = nltk.tokenize.sent_tokenize(text)

  for sentence in sentences:
    iter+= 1
    total_len = len(tokenizer.tokenize(sentence)) + length
    if total_len < maxtokens:
      comb += sentence
      length = total_len
      if iter == len(sentences):
        comblist.append(comb.strip())
    else:
      comblist.append(comb.strip())
      comb = sentence + " "
      length = len(tokenizer.tokenize(sentence))
  inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
  summ_input = []
  for input in inputs:
    output = model.generate(**input)
    summ_input.append(tokenizer.decode(*output, skip_special_tokens=True))
  tokenizer = AutoTokenizer.from_pretrained(sent_mod)
  model = AutoModelForSequenceClassification.from_pretrained(sent_mod)
  RobertaPos = []
  RobertaNeg = []
  RobertaNeu = []
  for i in range(len(summ_input)):
      text = summ_input[i]
      encoded_text = tokenizer(text, return_tensors='pt')
      output = model(**encoded_text)
      scores = output[0][0].detach().numpy()
      scores = softmax(scores)
      RobertaPos.append(scores[2])
      RobertaNeg.append(scores[0])
      RobertaNeu.append(scores[1])
  RobertaPositive = np.mean(RobertaPos)
  RobertaNegative = np.mean(RobertaNeg)
  RobertaNeutral = np.mean(RobertaNeu)
  return RobertaPositive, RobertaNegative, RobertaNeutral

In [14]:
def meeting(text):
  final = summary(text)[0]
  first_summary = summary(text)[1]
  RobertaPositive = sentiment(text)[0]
  RobertaNegative = sentiment(text)[1]
  RobertaNeutral = sentiment(text)[2]
  if final == first_summary:
    print(f'Meeting Summary: {final}\n-----------\nSentiment Analysis:\nPositive: {RobertaPositive}\nNegative: {RobertaNegative}\nNeutral: {RobertaNeutral}')
  elif final != first_summary:
    print(f'Headline: {final}\n----------\nMeeting Summary: {first_summary}\n-----------\nSentiment Analysis:\nPositive: {RobertaPositive}\nNegative: {RobertaNegative}\nNeutral: {RobertaNeutral}')


In [15]:
def meeting_b(text):
  text = text.strip()
  data=re.findall(regex, text)
  data = list(set(data))
  lst = []
  lsttext = []
  listsummary = []
  for i in range(len(data)):
    lst.append([])
    lsttext.append('')
  lines = text.splitlines( )
  for i in range(len(lines)):
    for j in range(len(data)):
      if lines[i].split()[0] == data[j] +':':
        lst[j].append(lines[i].replace(data[j] +':', '').strip())
  for i in range(len(lst)):
    for j in lst[i]:
      lsttext[i] = lsttext[i] + ' ' + j
  for i in range(len(lsttext)):
    lsttext[i] = lsttext[i].strip()
  for i in range(len(data)):
    listsummary.append('')
  for i in range(len(data)):
    listsummary[i] = summary(lsttext[i])[0]
  for i in range(len(data)):
    RobPos.append(sentiment(lsttext[i])[0])
    RobNeg.append(sentiment(lsttext[i])[1])
    RobNeu.append(sentiment(lsttext[i])[2])
  final = summary(text)[0]
  print(f'Headline: {final}\n----------\n')
  for i in range(len(data)):
    print(f'{data[i]}:\nTalking Points: {listsummary[i]}\nSentiment Analysis:\nPositive:{RobPos[i]}\nNegative:{RobNeg[i]}\nNeutral:{RobNeu[i]}\n--------------\n')

In [16]:
meeting(text)

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Headline:  The concept design meeting is taking place. There will be three presentations: the conceptual specification by Industrial Designer, the specification of the remote control and the design of the user interface. According to fashion watchers in Paris and Milan, fruits and vegetables will be the most important theme for clothes, shoes,
----------
Meeting Summary:  The concept design meeting is taking place. There will be three presentations: the conceptual specification by Industrial Designer, the specification of the remote control and the design of the user interface. The design of the remote control is as follows: banana shape with button on the side and a wheel on the top. The wheel sensor is standard for remote control, but the push button is expensive. The user interface is simple, innovative and easy to use. The designer and the industrial I don't think that this is really expensive. I was thinking to have a continuous light and you w when you press the on button you hav